<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=14d0c4d6766bff385452d4f1655c2ca497afbb483b75acc880a3485201c50992
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-26 13:45:46
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.43 C
-------------------
Today PnL: 1.31 L (0.92%)
Current PnL: -22.30 L (-14.61%)
CY Booked + Current PnL: -8.41 L (-5.51%)
-------------------
Total profit:  1.56 L
Total loss:  -23.86 L
-------------------
Total Booked + Current PnL: 18.57 L (14.78%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.68%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 87.71 L (61.12%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.56%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,2.76,-6.82,7.34,0.02,11939.0,-11902.0,162657.0,148.33,74.0,M-SC,7.94,234.0,-1.00,1.13,41.48,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.49,-13.68,15.96,0.09,13884.0,-13786.0,86991.0,101.92,52.0,M-SC,3.72,253.0,-0.99,0.61,14.29,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,2.04,8.27,10.55,19.69,21902.0,15858.0,207600.0,-2.32,52.0,X-MC,3.39,78.0,0.72,1.45,15.63,XY25,NTT,AC
53,NESTLEIND,1377.00,1.32,12.84,7.59,21.40,22732.0,34071.0,299497.0,4.67,59.0,X-LC,7.80,12.0,1.50,2.09,20.33,XY25,NTT,FMCG
35,ICICIGI,2252.93,0.07,6.73,12.62,20.19,23479.0,11726.0,186046.0,-16.25,50.0,X-MC,7.44,68.0,0.50,1.30,22.86,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SIEMENS,4671.50,4.15,-10.99,41.01,25.51,67933.0,-20445.0,165650.0,1.77,66.0,H-LC,1.89,49.0,-0.30,1.15,22.07,AR,ATH,ELECTRICAL
82,VBL,671.64,3.56,-6.06,44.42,35.67,131795.0,-19140.0,296702.0,-17.17,54.0,X-LC,5.97,4.0,-0.15,2.07,6.95,X40N,ATH,FMCG
64,SFL,1287.00,3.50,-39.60,109.66,26.63,173672.0,-103845.0,158373.0,19.88,36.0,M-SC,4.95,239.0,-0.60,1.10,3.50,XY24,NTT,MISC
7,ATULAUTO,844.00,3.33,-19.93,81.80,45.57,129117.0,-39287.0,157845.0,3413.51,47.0,M-SC,4.12,244.0,-0.30,1.10,11.91,XY24,NTT,AUTO
39,INDIGOPNTS,1408.00,2.76,-6.82,7.34,0.02,11939.0,-11902.0,162657.0,148.33,74.0,M-SC,7.94,234.0,-1.00,1.13,41.48,OX40N,NTT,PAINTS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-1.82,-27.72,97.61,42.84,45866.0,-18017.0,46989.0,-52.62,41.0,X-SC,37.47,83.0,-0.39,0.33,2.24,XY24,NTT,MISC
44,JCHAC,2282.00,-1.68,-24.77,32.91,-0.01,31642.0,-31659.0,96146.0,19573.91,32.0,M-SC,1.61,233.0,-1.00,0.67,8.87,OX40N,NTT,AC
62,ROUTE,2227.21,-1.48,-50.04,238.35,69.05,158462.0,-66583.0,66483.0,-59.60,36.0,H-SC,23.71,140.0,-0.42,0.46,2.53,SR,ATH,IT
38,INDIAMART,4810.62,-1.47,-0.10,106.93,106.72,131753.0,-122.0,123214.0,-50.63,38.0,H-SC,3.38,139.0,-0.00,0.86,23.26,AR,ATH,MISC
69,STARHEALTH,761.00,-1.09,-5.02,55.04,47.26,146969.0,-14101.0,267022.0,29.58,48.0,H-SC,15.50,171.0,-0.10,1.86,43.06,XY24,NTT,INSURANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-1.47,-0.10,106.93,106.72,131753.0,-122.0,123214.0,-50.63,38.0,H-SC,3.38,139.0,-0.00,0.86,23.26,AR,ATH,MISC
86,ZYDUSLIFE,1286.17,1.49,-0.63,36.49,35.63,75994.0,-1315.0,208259.0,-14.53,44.0,H-MC,4.18,119.0,-0.02,1.45,15.94,AR,ATH,PHARMA


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.21,-4.37,117.42,107.91,165779.0,-6459.0,141185.0,-22.17,37.0,M-SC,10.47,216.0,-0.04,0.98,0.96,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,2.76,-6.82,7.34,0.02,11939.0,-11902.0,162657.0,148.33,74.0,M-SC,7.94,234.0,-1.00,1.13,41.48,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.49,-13.68,15.96,0.09,13884.0,-13786.0,86991.0,101.92,52.0,M-SC,3.72,253.0,-0.99,0.61,14.29,OX40N,NTT,DURABLES
47,KANSAINER,340.00,0.29,-22.68,46.75,13.47,97479.0,-61155.0,208512.0,-68.12,27.0,H-SC,1.28,158.0,-0.63,1.45,6.02,XY24,NTT,PAINTS
67,SIS,528.00,0.68,-23.35,58.20,21.26,49532.0,-25926.0,85106.0,2008.81,50.0,H-SC,3.88,166.0,-0.52,0.59,14.99,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.0,1.51,2.27,68.44,72.27,118605.0,3850.0,173298.0,-12.43,59.0,M-LC,4.41,99.0,0.03,1.21,8.59,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,1.51,2.27,68.44,72.27,118605.0,3850.0,173298.0,-12.43,59.0,M-LC,4.41,99.0,0.03,1.21,8.59,XR,NTT,IT
38,INDIAMART,4810.62,-1.47,-0.10,106.93,106.72,131753.0,-122.0,123214.0,-50.63,38.0,H-SC,3.38,139.0,-0.00,0.86,23.26,AR,ATH,MISC
86,ZYDUSLIFE,1286.17,1.49,-0.63,36.49,35.63,75994.0,-1315.0,208259.0,-14.53,44.0,H-MC,4.18,119.0,-0.02,1.45,15.94,AR,ATH,PHARMA
25,FINCABLES,1641.55,0.21,-4.37,117.42,107.91,165779.0,-6459.0,141185.0,-22.17,37.0,M-SC,10.47,216.0,-0.04,0.98,0.96,OX40N,ATH,CABLES
84,WHIRLPOOL,2270.00,-0.53,-4.71,90.92,81.92,99456.0,-5410.0,109388.0,-44.16,35.0,M-SC,3.79,236.0,-0.05,0.76,29.50,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.55,-35.44,109.43,35.20,90899.0,-45604.0,83066.0,8.88,24.0,X-SC,13.04,92.0,-0.50,0.58,0.04,X40,NTT,FOOTWEAR
46,JSWINFRA,342.00,0.43,-9.55,27.40,15.23,49650.0,-19143.0,181204.0,-22.54,25.0,X-MC,6.80,66.0,-0.39,1.26,19.61,X40N,NTT,REALTY
76,TMPV,600.00,1.63,-15.12,67.50,42.18,158369.0,-41789.0,234621.0,-24.30,28.0,X-LC,1.07,3.0,-0.26,1.64,1.63,XY24,NTT,AUTO
9,BAJAJHFL,181.50,0.09,-14.08,72.77,48.45,125370.0,-28224.0,172282.0,-20.30,29.0,X-MC,7.08,64.0,-0.23,1.20,0.09,X40N,ATH,FINANCE
55,PGHH,17907.65,-0.34,-4.54,40.12,33.76,76910.0,-9120.0,191700.0,-32.64,33.0,X-MC,3.55,57.0,-0.12,1.34,0.42,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.0,0.40,-1.68,12.33,10.44,24262.0,-3364.0,196774.0,-41.15,42.0,X-LC,0.89,5.0,-0.14,1.37,3.79,X40,NTT,FMCG
76,TMPV,600.0,1.63,-15.12,67.50,42.18,158369.0,-41789.0,234621.0,-24.30,28.0,X-LC,1.07,3.0,-0.26,1.64,1.63,XY24,NTT,AUTO
3,ACC,3906.0,-0.18,-21.59,109.35,64.15,204025.0,-51371.0,186580.0,-54.13,53.0,X-SC,1.62,82.0,-0.25,1.30,4.66,XY24,BTT,CEMENT
36,ICICIPRULI,790.0,1.35,2.87,27.32,30.97,49669.0,5068.0,181806.0,-21.38,56.0,X-MC,1.69,75.0,0.10,1.27,15.79,X40,ATH,INSURANCE
66,SIEMENS,4671.5,4.15,-10.99,41.01,25.51,67933.0,-20445.0,165650.0,1.77,66.0,H-LC,1.89,49.0,-0.30,1.15,22.07,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.55,-35.44,109.43,35.20,90899.0,-45604.0,83066.0,8.88,24.0,X-SC,13.04,92.0,-0.50,0.58,0.04,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,0.09,-14.08,72.77,48.45,125370.0,-28224.0,172282.0,-20.30,29.0,X-MC,7.08,64.0,-0.23,1.20,0.09,X40N,ATH,FINANCE
55,PGHH,17907.65,-0.34,-4.54,40.12,33.76,76910.0,-9120.0,191700.0,-32.64,33.0,X-MC,3.55,57.0,-0.12,1.34,0.42,X40,ATH,FMCG
54,PAGEIND,51605.08,0.91,-5.01,32.71,26.07,50877.0,-8200.0,155540.0,-30.06,38.0,X-MC,9.19,55.0,-0.16,1.08,0.91,X40,ATH,APPARELS
76,TMPV,600.00,1.63,-15.12,67.50,42.18,158369.0,-41789.0,234621.0,-24.30,28.0,X-LC,1.07,3.0,-0.26,1.64,1.63,XY24,NTT,AUTO


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-1.82,-27.72,97.61,42.84,45866.0,-18017.0,46989.0,-52.62,41.0,X-SC,37.47,83.0,-0.39,0.33,2.24,XY24,NTT,MISC
60,RELAXO,1176.00,2.69,-46.78,186.20,52.31,143838.0,-67911.0,77249.0,-43.37,38.0,X-SC,4.10,91.0,-0.47,0.54,3.40,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.55,-35.44,109.43,35.20,90899.0,-45604.0,83066.0,8.88,24.0,X-SC,13.04,92.0,-0.50,0.58,0.04,X40,NTT,FOOTWEAR
22,DIXON,18931.72,2.60,-5.11,28.13,21.58,37406.0,-7164.0,132975.0,-52.17,37.0,X-MC,6.71,56.0,-0.19,0.93,17.72,X40N,ATH,IT
52,MEDANTA,1486.00,1.10,5.48,17.63,24.08,23608.0,6960.0,133910.0,-3.08,51.0,X-SC,5.18,89.0,0.29,0.93,26.17,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,1.72,-13.79,38.35,19.28,114383.0,-47694.0,298262.0,-24.92,64.0,X-LC,7.35,1.0,-0.42,2.08,10.26,X40,ATH,IT
41,INFY,2275.00,1.32,5.57,46.70,54.87,154259.0,17432.0,330320.0,-17.80,60.0,X-LC,2.86,2.0,0.11,2.30,14.32,X40,BTT,IT
76,TMPV,600.00,1.63,-15.12,67.50,42.18,158369.0,-41789.0,234621.0,-24.30,28.0,X-LC,1.07,3.0,-0.26,1.64,1.63,XY24,NTT,AUTO
82,VBL,671.64,3.56,-6.06,44.42,35.67,131795.0,-19140.0,296702.0,-17.17,54.0,X-LC,5.97,4.0,-0.15,2.07,6.95,X40N,ATH,FMCG
43,ITC,452.00,0.40,-1.68,12.33,10.44,24262.0,-3364.0,196774.0,-41.15,42.0,X-LC,0.89,5.0,-0.14,1.37,3.79,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,1.54,-37.04,111.07,32.89,53991.0,-28596.0,48610.0,-700.55,64.0,L-MC,5.59,259.0,-0.53,0.34,33.87,XR,NTT,BANKS
6,ASIANTILES,137.00,0.56,-12.23,106.98,81.67,88087.0,-11470.0,82340.0,7405.56,54.0,L-SC,19.05,271.0,-0.13,0.57,59.46,XR,NTT,CERAMICS
68,SONACOMS,806.63,0.73,-12.65,59.74,39.54,52790.0,-12793.0,88366.0,-31.70,65.0,M-SC,6.63,220.0,-0.24,0.62,24.61,AR,ATH,AUTO
51,MASFIN,398.61,0.05,-5.76,29.50,22.05,27240.0,-5640.0,92340.0,-18.70,48.0,H-SC,7.38,164.0,-0.21,0.64,34.22,XR,ATH,FINANCE
70,SURYODAY,216.00,1.47,-18.38,50.73,23.03,74150.0,-32905.0,146166.0,60.09,51.0,H-SC,10.18,167.0,-0.44,1.02,45.03,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,2.76,-6.82,7.34,0.02,11939.0,-11902.0,162657.0,148.33,74.0,M-SC,7.94,234.0,-1.00,1.13,41.48,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,-0.24,0.28,20.61,20.95,52034.0,704.0,252472.0,-0.77,74.0,X-LC,13.93,31.0,0.01,1.76,36.02,X40,ATH,PAINTS
40,INDUSINDBK,1800.00,1.54,-37.04,111.07,32.89,53991.0,-28596.0,48610.0,-700.55,64.0,L-MC,5.59,259.0,-0.53,0.34,33.87,XR,NTT,BANKS
79,UNITDSPR,1644.00,1.36,6.50,13.39,20.77,33390.0,15222.0,249366.0,-2.78,64.0,X-MC,3.71,62.0,0.46,1.74,13.87,X40N,NTT,BREWERIES
0,5PAISA,593.00,1.53,-35.52,75.11,12.90,105813.0,-77618.0,140878.0,107.93,57.0,H-SC,13.81,161.0,-0.73,0.98,17.14,OX40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.78
1,25,44.61
2,50,75.52


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.58
MC    29.25
LC    25.18
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.88
X40      21.16
X40N     11.93
XR        9.60
XY25      9.09
AR        9.08
OX40N     7.68
X200      1.84
SR        0.97
MH        0.78
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.87
X-MC    22.79
X-LC    20.01
M-SC    12.07
X-SC     8.24
H-MC     4.75
H-LC     2.99
L-SC     1.40
M-MC     1.37
M-LC     1.21
L-LC     0.97
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.79,-4.53,38.34
IT,13.74,-14.67,73.86
FINANCE,9.41,-14.77,63.27
MISC,7.42,-22.01,71.99
ELECTRICAL,5.98,-9.19,49.19
PAINTS,5.96,-7.90,23.93
INSURANCE,4.43,0.42,34.67
PHARMA,3.89,-0.27,32.36
AUTO,3.36,-19.52,70.77


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3035542.0,21
XR,1284028.0,13
AR,1271389.0,10
X40,1003550.0,14
X40N,737115.0,9
OX40N,688566.0,10
XY25,337860.0,6
SR,280433.0,2
MH,72454.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3554588.0,25
M-SC,1325371.0,15
X-MC,1214778.0,16
X-LC,892137.0,11
X-SC,737719.0,8
H-MC,396681.0,3
L-SC,260194.0,3
H-LC,127869.0,2
M-LC,118605.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1218183.0      6
           AR         873285.0      5
           XR         789986.0      7
M-SC       XY24       773172.0      6
X-MC       X40        437786.0      7
X-LC       X40        392153.0      5
X-MC       XY24       354737.0      3
H-SC       OX40N      320247.0      4
X-SC       X40N       290609.0      3
M-SC       OX40N      280690.0      5
H-SC       SR         280433.0      2
X-SC       XY24       273499.0      3
X-MC       X40N       245816.0      4
X-LC       XY24       226813.0      2
H-MC       AR         207543.0      2
X-LC       X40N       200690.0      2
H-MC       XY24       189138.0      1
X-MC       XY25       176439.0      2
X-SC       X40        173611.0      2
L-SC       XR         172565.0      2
M-SC       XR         148881.0      2
           AR         122628.0      2
M-LC       XR         118605.0      1
L-SC       OX40N       87629.0      1
X-LC       XY25        72481.0      2
H-SC       MH          72454.0      1
H-LC       AR          67933.0      1
           X200        59936.0      1
L-MC       XR          53991.0      1
M-MC       XY25        48840.0      1
L-LC       XY25        40100.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
